# 7-Slot Nerdle Solver Test
We prove (by brute-force) that you can always solve mini-Nerdle in at most $4$ guesses regardless of the starting expression, provided you use the optimal strategy. The worst start having repeating numbers and thus less information, e.g. `10-5=5`. The best start has all different numbers: `28/7=4`, which needs at most $3$ guesses and $2.65 \pm 0.5$ guesses.

In [1]:
%load_ext autoreload
%autoreload 2

import collections
import itertools
import numpy as np
import matplotlib.pyplot as plt

import nerdle
import score as s
import generator
from nerdle import Hint, NerdleData

In [2]:
# Mini-Nerdle.
NUM_SLOTS = 8
SCORE_DICT_FILE = "nerdle{}".format(NUM_SLOTS) 

solver_data = nerdle.create_solver_data(NUM_SLOTS, SCORE_DICT_FILE)

0 / 17723 (0.0%) completed
886 / 17723 (5.0%) completed
1772 / 17723 (10.0%) completed
2658 / 17723 (15.0%) completed
3544 / 17723 (20.0%) completed
4430 / 17723 (25.0%) completed
5316 / 17723 (30.0%) completed
6202 / 17723 (35.0%) completed
7088 / 17723 (40.0%) completed
7974 / 17723 (45.0%) completed
8860 / 17723 (50.0%) completed
9746 / 17723 (55.0%) completed
10632 / 17723 (60.0%) completed
11518 / 17723 (65.0%) completed
12404 / 17723 (70.0%) completed
13290 / 17723 (75.0%) completed
14176 / 17723 (80.0%) completed
15062 / 17723 (85.0%) completed
15948 / 17723 (90.0%) completed
16834 / 17723 (95.0%) completed
17720 / 17723 (100.0%) completed


In [3]:
d = solver_data.score_dict
print(len(d), len(solver_data.answers))
for key in list(d.keys())[:10]:
    print(key, "".join(map(str, key[0])) + "=" + str(key[1]), len(d[key]))

17723 17723
204/4=51 2=0 17723
19+49=68 1=9 17723
27+67=94 2=7 17723
44-12=32 4=4 17723
68-26=42 6=8 17723
5+8-10=3 5=+ 17723
65+19=84 6=5 17723
69-8*8=5 6=9 17723
17+79=96 1=7 17723
744/8=93 7=4 17723


## Example Usage

In [9]:
score = nerdle.score_guess("10-43=66", "12+34=56")
print(score, 
      s.score_to_hint_string(score, NUM_SLOTS),
      s.score_to_hints(score, NUM_SLOTS) == [Hint.CORRECT, Hint.INCORRECT, Hint.INCORRECT, Hint.MISPLACED,
                               Hint.MISPLACED, Hint.CORRECT, Hint.INCORRECT, Hint.CORRECT]
     )

18049 +--??+-+ True


In [11]:
# A good initial guess significantly reduces the number of answers. In this case, from
# 206 to 10.
solver = nerdle.NerdleSolver(solver_data)
guess_history, hint_history, answer_size_history = solver.solve("5+6+6=17", initial_guess= "9*8-7=65", debug=True)

guess 9*8-7=65 score ----?+?? answers 71
guess 11+46=57 score ?-?-++?+ answers 2
guess 5+6+6=17 score ++++++++ answers 1


In [19]:
def hint_generator(guess):
    if guess == "9*8-7=65":
        return s.hints_to_score([Hint.INCORRECT, Hint.INCORRECT, Hint.INCORRECT, Hint.INCORRECT,
                Hint.INCORRECT, Hint.CORRECT, Hint.CORRECT, Hint.MISPLACED])
    elif guess == "":
        return s.hints_to_score([Hint.INCORRECT, Hint.INCORRECT, Hint.INCORRECT, Hint.INCORRECT,
                Hint.INCORRECT, Hint.CORRECT, Hint.CORRECT, Hint.MISPLACED])
    elif guess == "":
        return s.hints_to_score([Hint.INCORRECT, Hint.INCORRECT, Hint.INCORRECT, Hint.INCORRECT,
                Hint.INCORRECT, Hint.CORRECT, Hint.CORRECT, Hint.MISPLACED])

In [20]:
# A good initial guess significantly reduces the number of answers. In this case, from
# 206 to 10.
solver = nerdle.NerdleSolver(solver_data)
guess_history, hint_history, answer_size_history = solver.solve_adversary(hint_generator, initial_guess= "9*8-7=65", debug=True)

guess 9*8-7=65 score -----++?
guess 9*8-7=65 score -----++? answers 70


TypeError: 'NoneType' object cannot be interpreted as an integer

## Benchmark
This is a fast in-memory dict implementation.

In [6]:
%time guess_history, hint_history, answer_size_history = solver.solve("5+6+6=17", initial_guess="9*8-7=65")

CPU times: user 31.5 s, sys: 38.9 s, total: 1min 10s
Wall time: 1min 57s


## Initial Guess Optimization
Assuming something with a lot of different numbers and operators is best.

In [17]:
start = "9*8-7=65"
solutions = [nerdle.NerdleSolver(solver_data).solve(answer, initial_guess=start) 
             for answer in solver_data.answers]               
n = np.array([len(solution[0]) for solution in solutions])
num_answers = len(solver_data.answers)
compression_ratio = num_answers / np.array([solution[2][0] for solution in solutions])
print(np.mean(compression_ratio), np.std(compression_ratio))

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
ax.hist(n);
ax.set_title("#Guesses for start {}".format(start));

ax = axs[1]
ax.hist(compression_ratio);
ax.set_title("Compression Ratio Distribution, start {}".format(start));

KeyboardInterrupt: 